In [5]:
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import graphlab as gl
from graphlab import SGraph, Vertex, Edge, degree_counting, connected_components, SFrame
import re
import slate
import cPickle as pickle

In [7]:
from fenaroli_parser.build_network import build_weighted_graph, extract_backbone

In [8]:
with open('./data/first_ing_comp_dict.pkl' ,'r') as f:
    ing_flav_dict = pickle.load(f)
with open('./data/recipes.pkl' ,'r') as f:
    recipes = pickle.load(f)

In [28]:
flav_network = SGraph()
vert_list = []
edge_list = []
ingrds_not_seen = ing_flav_dict.keys()

for ingr, comp_list in ing_flav_dict.iteritems():
    ingrds_not_seen.remove(ingr)
    vert_list.append(Vertex(ingr, attr={'deg':0}))
    for ingrd in ingrds_not_seen:
        weight = len(set(ing_flav_dict[ingrd]).intersection(set(comp_list)))
        if weight > 0:
            edge_list.append(Edge(ingr, ingrd, attr={'weight':weight}))
    vert_list.append(Vertex(ingr))

flav_network = flav_network.add_vertices(vert_list)
flav_network = flav_network.add_edges(edge_list)

In [46]:
flvn = SGraph()
new_node_list = flav_network.vertices.fillna('deg', 0)
flvn = flvn.add_vertices(new_node_list).add_edges(edge_list)

[ERROR] GraphLab Canvas: (<type 'exceptions.RuntimeError'>, RuntimeError('Communication Failure: 65. ',), <traceback object at 0x1186587a0>)


RuntimeError: Communication Failure: 65. 

In [30]:
flvn.show()

Canvas is updated and available in a tab in the default browser.


In [31]:
flav_network.summary()

{'num_edges': 94715, 'num_vertices': 2659}

In [9]:
import cProfile

In [9]:
def get_range(dictionary, begin, end):
  return dict(itertools.islice(dictionary.iteritems(), begin, end+1)) 

In [10]:
len(ing_flav_dict)/10

265.9

In [18]:
# ing_flav_dict_sub = get_range(ing_flav_dict,0,1000)

In [10]:
fnet, node_list, edge_list = build_weighted_graph(ing_flav_dict)

[INFO] 1452034161 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /usr/local/lib/python2.7/site-packages/certifi/cacert.pem
1452034161 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This trial license of GraphLab Create is assigned to drschwenk@gmail.com and will expire on January 16, 2016. Please contact trial@dato.com for licensing options or to request a free non-commercial license for personal or academic use.

[INFO] Start server at: ipc:///tmp/graphlab_server-48941 - Server binary: /usr/local/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1452034161.log
[INFO] GraphLab Server Version: 1.7.1


In [24]:
backbone_a_1 = extract_backbone(fnet, node_list, edge_list, 2)

In [25]:
backbone_a_1[0]

E(macadamia nuts -> rice, {'weight': 3})

In [29]:
edge_dict = defaultdict(int)
for edge in backbone_a_1:
    n1 = edge.dst_vid
    n2 = edge.src_vid
    key = n1+', '+n2
    edge_dict[key] = edge.attr['weight']

edge_dict

defaultdict(int,
            {'apple, mandar': 4,
             'soybean, meats': 5,
             'rose apple, melon': 4,
             'tomato, mandar': 6,
             'identified, strawberry': 3,
             'hop oil, tea': 25,
             'kelp, avocado': 3,
             'apple, apple brandy': 9,
             'bilberry, cassia leaf': 5,
             'krill, milk': 4,
             'vinegar, apricot': 7,
             'boiled egg, cocoa': 6,
             'strawberry, butter': 24,
             'also, yogurt': 5,
             'crab, beef': 4,
             'myrtle leaf, licorice': 3,
             'asparagus, cooked': 3,
             'cocoa, papaya': 30,
             'sweet corn, caviar': 3,
             'yogurt, honey': 4,
             'fish oil, plum': 3,
             'apple brandy, starfruit': 3,
             'cloudberry, guava': 7,
             'grapes, spineless monkey orange': 6,
             'tea, nuts': 3,
             'elderberry, papaya': 7,
             'white, coffee': 3,
    

In [19]:
backbone_a_1[0]

E(macadamia nuts -> rice, {'weight': 3})

In [30]:
with open('./backbone_edges_dict.pkl', 'w') as f:
    pickle.dump(edge_dict, f)

In [37]:
%prun backbone_a_2 = extract_backbone(fnet, 1)

TypeError: extract_backbone() takes exactly 4 arguments (2 given)

In [12]:
backbone_a_2.show()

Canvas is accessible via web browser at the URL: http://localhost:51730/index.html
Opening Canvas in default web browser.


In [8]:
backbone_a_2.save('./fn_bb_a2')

In [10]:
backbone_a_1 = gl.load_sgraph('./fn_bb_a1/')